In [211]:
from sage.all import *
import numpy as np
import random

def kernPause(a1,a2):
    return  1*(a1==a2)

def kernPitch(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)**2/(a*b)

def kernVolume(v1,v2):
    #return kernJacc(v1,v2)
    return min(v1,v2) #/max(v1,v2)

def kernDuration(d1,d2):
    return min(d1,d2)

def create_random_graph(n):
    G = Graph(loops=False)
    G.add_vertex(1-1)
    for k in range(2,n+1):
        vert = [v for v in G.vertices()]
        G.add_vertex(k-1)
        for v in vert:
            prob = 1.0/v*k/sigma(k)
            p = randint(1,100)/100.0
            #print(prob,p)
            if p <= prob and k%(v+1)==0:
                G.add_edge(v,k-1)
    return G

def create_divisor_graph(n):
    G = DiGraph(loops=True)
    divs = divisors(n)
    for d in divs:
        G.add_vertex(divs.index(d))
    for u in divs:
        for v in divs:
            if is_prime(u%v):
                G.add_edge(divs.index(u),divs.index(v))
    return G

def kernAdd(t1,t2,alphaPitch=0.25):
    pitch1,volume1,isPause1 = t1
    pitch2,volume2,isPause2 = t2
    #return 1.0/3*(1-alphaPitch)*kernPause(isPause1,isPause2)+alphaPitch*kernPitch(pitch1,pitch2)+1.0/3*(1-alphaPitch)*kernDuration(duration1,duration2)+1.0/3*(1-alphaPitch)*kernVolume(volume1,volume2)
    apa = alphaPitch["pause"]
    api = alphaPitch["pitch"]
    avo = alphaPitch["volume"]
    #return kernPause(isPause1,isPause2)*kernPitch(pitch1,pitch2)*kernVolume(volume1,volume2)

    if np.abs(apa+api+avo-1)<10**-5:
        return apa*kernPause(isPause1,isPause2)+api*kernPitch(pitch1,pitch2)+avo*kernVolume(volume1,volume2)
    else:
        return None

def kern0(zz0,alphaPitch={"pitch":1,"volume":2,"pause":3}):
    return lambda t1,t2: kernAdd(zz0[int(t1[0])],zz0[int(t2[0])],alphaPitch)

def kern(alphaPitch={"pitch":1,"volume":2,"pause":3}):
    return lambda t1,t2: kernAdd(t1,t2,alphaPitch)



def distKern1(x,y,alphaPitch={"pitch":1,"volume":2,"pause":3}):
    #print(alphaPitch)
    return np.sqrt(2-2*kern(alphaPitch)(x,y))

def distKern(kern):
    return lambda a,b : np.sqrt(kern(a,a)+kern(b,b)-2*kern(a,b))

import music21 as m21
from itertools import product

durlist = [[sum([(power(2,(n-i))) for i in range(d+1)]) for n in range(-8,3+1)] for d in range(2)]
durationslist = []
for dl in durlist:
    durationslist.extend([x for x in dl if x<=1.5])
print(durationslist)

def findNearestDuration(duration,durationslist):
    return sorted([(abs(duration-nv),nv) for nv in durationslist])[0][1]

def parse_file(xml):
    xml_data = m21.converter.parse(xml)
    score = []
    for part in xml_data.parts:
        parts = []
        for note in part.recurse().notesAndRests:
            if note.isRest:
                start = note.offset
                duration = float(note.quarterLength)#/4.0
                vol = 32 #note.volume.velocity
                pitch = 60
                parts.append([pitch,findNearestDuration(duration,durationslist),vol,True])
            else:
                #print(note)
                start = note.offset
                duration = float(note.quarterLength)#/4.0
                pitch = note.pitch.midi
                #print(pitch,duration,note.volume)
                vol = note.volume.velocity
                if vol is None:
                    vol = int(note.volume.realized * 127)
                parts.append([pitch,findNearestDuration(duration,durationslist),vol,False])    
        score.append(parts)        
    return score


def writePitches(fn,inds,tempo=82,instrument=[0,0],add21=True,start_at= [0,0],durationsInQuarterNotes=False):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = start_at
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause = inds[k][i]
            print("writePitches",pitch,duration,volume,isPause)
            track = k
            channel = k
            if not durationsInQuarterNotes:
                duration = 4*duration#*maxDurations[k] #findNearestDuration(duration*12*4)            
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(floor(pitch))
                if add21:
                    pitch += 21
                
                MyMIDI.addNote(track, channel, int(pitch), (times[k]) , float(duration), int(volume))
                times[k] += duration*1.0  
                print("times > ", pitch,times[k],duration,volume,isPause)    
            else:
                times[k] += duration*1.0
                print("pause - times > ", pitch,times[k],duration,volume,isPause)    
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")  


def run_length_row(row_of_01):
    from itertools import groupby
    ar = row_of_01
    return [(k, sum(1 for i in g)) for k,g in groupby(ar)]

def int_comp_row(row_of_01,by=8):
    ll = divide_row_by(row_of_01,by=by)
    ss = []
    for l in ll:
        rl = run_length_row(l)
        ss.append([v for k,v in rl])
    return list(ss)    

def divide_row_by(row,by):
    ll = []
    n = len(row)
    m = n//by
    #print(m)
    for k in range(m):
        ll.append(row[k*by:((k+1)*by)])
    return list(ll)    

def generateNotes(pitchlist,alphaPitch={"pitch":1,"volume":2,"pause":3},shuffle_notes=True):
    from itertools import product
    from music21 import pitch
    #pitchlist = [p for p in list(range(60-1*octave*12,60+24-1*octave*12))]
    #distmat = np.array(matrix([[np.sqrt(2*(1.0-kernPitch(x,y))) for x in pitchlist] for y in pitchlist]))
    #permutation,distance = tspWithDistanceMatrix(distmat,exact=False)
    #pitchlist = [pitchlist[permutation[k]] for k in range(len(pitchlist))]
    print([pitch.Pitch(midi=int(p)) for p in pitchlist])
    #durationlist = [n for n in durs]
    #if len(durs)>2:
    #    distmat = np.array(matrix([[np.sqrt(2*(1.0-kernDuration(x,y))) for x in durationlist] for y in durationlist]))
    #    permutation,distance = tspWithDistanceMatrix(distmat)
    #    durationlist = [durationlist[permutation[k]] for k in range(len(durationlist))]
    #print(durationlist)
    volumelist = vols = [(128//8)*(k+1) for k in range(8)] #[x*127 for x in [1.0/6.0,1.0/3.0,1.0/2.0,2.0/3.0 ]]
    print(volumelist)
    #distmat = np.array(matrix([[np.sqrt(2*(1.0-kernVolume(x,y))) for x in volumelist] for y in volumelist]))
    #permutation,distance = tspWithDistanceMatrix(distmat)
    #volumelist = [volumelist[permutation[k]] for k in range(len(volumelist))]
    print(volumelist)
    pauselist = [False,True]
    ll = list(product(pauselist,volumelist,pitchlist))
    if shuffle_notes:
        shuffle(ll)
    #distmat = np.array(matrix([[distKern(x,y,alphaPitch) for x in ll] for y in ll]))
    #np.random.seed(43)
    #permutation,distance = tspWithDistanceMatrix(distmat,exact=False)
    #ll = [ll[permutation[k]] for k in range(len(ll))]
    print(len(ll))
    #print(ll)
    pitches = [p[2] for p in ll]
    #durations = [d[0] for d in ll]
    volumes = [v[1] for v in ll]
    isPauses = [p[0] for p in ll]
    #print(pitches)
    return pitchlist,volumelist,pauselist

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)

def get_knn_model(for_list,kern=kernPitch):
    #notes = np.array([[x*1.0 for x in n] for n in notes])
    from sklearn.neighbors import NearestNeighbors
    np.random.seed(0)
    #nbrs = NearestNeighbors( algorithm='ball_tree',metric=distKern(kern(zz0,alphaPitch=alphaPitch))).fit([[r] for r in range(len(zz0))])
    M = matrix([[float(kern(a,b)) for a in for_list] for b in for_list],ring=RDF)
    #print(M)
    Ch = np.array(M.cholesky())
    nbrs = NearestNeighbors().fit(Ch)
    return nbrs, Ch

def findBestMatches(nbrs,new_row,n_neighbors=3):
    distances,indices = nbrs.kneighbors([np.array(new_row)],n_neighbors=n_neighbors)
    dx = sorted(list(zip(distances[0],indices[0])))
    #print(dx)
    indi = [d[1] for d in dx]
    #print(indi)
    #print(distances)
    #distances,indices = nbrs.query([np.array(new_row)],k=n_neighbors)
    return indi


from itertools import groupby

def get_flattened_durations_from_graph(G,permutation_of_vertices,sorted_reverse,by=16,shuffled=True):
    zz = sorted(list(zip(G.degree_sequence(),G.vertices())),reverse=sorted_reverse)
    #print(sorted(zz))
    if shuffled:
        shuffle(zz)
    A = G.adjacency_matrix(vertices = permutation_of_vertices)
    ll = []
    for row in A:
        #print(row)
        ll.extend([xx/by for xx in x] for x in (int_comp_row(row,by=by)))
    ss = []
    for l in ll:
        ss.extend(l)
    return ss    

def get_bitstring_from_durations(durs,denom=None):
    # convert reciprocal into rational number:
    # take the denominator
    # take the gcd of all denominators
    qq = ([QQ(d) for d in durs])
    dd = [q.denominator() for q in qq]
    D = denom if not denom is None else lcm(dd)
    nn = [q.numerator()*D/q.denominator() for q  in qq]
    xx = []

    print(D,nn)
    y = 0
    for n in nn:
        xx.extend(n*[y])
        y = 1*(not y)
    return xx,D    
    
 
def get_durations_from_graph(G,sorted_reverse,by=16,shuffled=True):
    zz = sorted(list(zip(G.degree_sequence(),G.vertices())),reverse=sorted_reverse)
    #print(sorted(zz))
    if shuffled:
        shuffle(zz)
    A = G.adjacency_matrix(vertices = [z[1] for z in zz])
    ss = []
    for row in A:
        print(row)
        xx = ((int_comp_row(row,by=by)))
        ll = []
        for x in xx: 
            ll.extend([xs/by for xs in x])
        ss.append(ll)  
    return ss    

def lee(x,p):
    return min(x%p, (p-x)%p)

def func(s):
    NBars = 60
    if s <= NBars//3:
        return int(s**2/(NBars//3)**2*8)+2
    elif s > NBars//3 and s < 2*NBars//3:
        return int((s-NBars//3)**2/(NBars//3)**2*8)+2
    else:
        return int((s-2*NBars//3)**2/(NBars//3)**2*8)+2

[1/256, 1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 3/512, 3/256, 3/128, 3/64, 3/32, 3/16, 3/8, 3/4, 3/2]


In [238]:
# flattened durations
by = 8
N = by*2*3
#G = graphs.RandomBarabasiAlbert(N,N//3) #create_random_graph(N)
#G = graphs.RandomIntervalGraph(N)
#G = create_divisor_graph(N)
#G = graphs.RandomGNM(N, 400, dense=False, seed=None)
pitches,volumes,isPauses = list(generateNotes(pitchlist=range(15,128-15),shuffle_notes=False))
#print(pitches)
durations = [1/16,1/8,1/4,1/2,1]
#print(len(notes))
#shuffle(notes)
nbrsPitch, chPitch = get_knn_model(pitches,kernPitch)
nbrsVolume, chVolume = get_knn_model(volumes,kernVolume)
#nbrsDuration, chDuration = get_knn_model(durations,kernDuration)
nbrsPause, chPause = get_knn_model(isPauses,kernPause)


scores = parse_file("./Piano_and_Cello.mid")


pitches0 = [[n[0] for n in notes] for notes in scores]
vols0 = [[n[2] for n in notes] for notes in scores]
pauses0 = [[n[3] for n in notes] for notes in scores]
durs0 = [[n[1] for n in notes] for notes in scores]
durs0 = [30*(4*[1/4]+8*[1/8]),30*(2*[1/2]+4*[1/4])]

#durs0 =[ 5*(
#           2*(16*[1/16]+4*[4/16]+[1/16,2/16,1/16,4/16]*2)
#          +3*( 8*[2/16]+16*[1/16]+[1/16,2/16,1/16,4/16]*3)
#          )+
#         6*(
#           3*(16*[1/16]+4*[4/16]+[1/16,2/16,1/16,4/16]*3)
#          +2*( 8*[2/16]+16*[1/16]+[1/16,2/16,1/16,4/16]*2)
#         )
#       ]

#8     CyclicPermutationGroup(8)                   Cyclic
#8     D1 = CyclicPermutationGroup(4)
#      D2 = CyclicPermutationGroup(2)
#      G = direct_product_permgroups([D1,D2])      Abelian, non-cyclic
#8     D1 = CyclicPermutationGroup(2)
#      D2 = CyclicPermutationGroup(2)
#      D3 = CyclicPermutationGroup(2)
#      G = direct_product_permgroups([D1,D2,D3])   Abelian, non-cyclic
#8     DihedralGroup(4)                            Non-abelian
#8     QuaternionGroup()                           Quaternions, also DiCyclicGroup(2)
#gr = AbelianGroup([2,2,2])

D1 = CyclicPermutationGroup(2)
D2 = CyclicPermutationGroup(2)
D3 = CyclicPermutationGroup(4)
#gr = direct_product_permgroups([D1,D2,D3]) 
gr = CyclicPermutationGroup(4) #16)8)
#gr = QuaternionGroup()

print(str(gr))

for g in gr:
    M = (g.matrix())
    print("---")
    print(M)

nVoices = gr.order()
durs = []
iinds = []
#or nv in range(nVoices):
for g in gr:
    M = g.matrix()
    for d1 in durs0:
        print(d1)
        #bs,D1 = get_bitstring_from_durations(d1,denom=32)
        bs,D1 = get_bitstring_from_durations(d1,denom=8)
        print(D1,"len(bs)=",len(bs))
        bbs = []
        jq = len(bs)//gr.order()
        jr = len(bs)%gr.order()
        print("jq,jr",jq,jr)
        for j in range(jq):
            v = vector(bs[j*gr.order():(j+1)*gr.order()])
            mv = [x for x in (M*v)]
            bbs.extend(mv)
        bbs.extend(bs[(j+1)*gr.order():])    
        d2 = [r[1]/D1 for r in run_length_row(bbs)]
        durs.append(d2)
        print("bs==bbs",bs==bbs,d2==d1)
        iinds.append([])





pitches1 = []
volumes1 = []
durations1 = []
#pauses1 = []

vpitches = []

pitchSet = set([])
pprimes = [4,3,4,3,2,3,2,3]
for d in range(len(durs)):
    pitchBuffer = []
    lenBuffers = [30]
    #pitches1 = []
    #volumes1 = []
    #durations1 = []
    #pauses1 = []
    startPitch = chPitch[0]
    startVolume = chVolume[0]
    #startPause = chPause[0]
    s = 0
    vpitches.append([])
    for k in range(len(durs[d])):
        duration1 = durs[d][k]
        s+=duration1
        vPitch  =  (k+d)%pprimes[d]+1 
        if d>=len(durs)-3:
            vPitch = int(sum(Integer(k+d+1).digits(5)))+1
        vVolume =  int(sum(Integer(k+d+1).digits(2)))+1
        #vPitch  =  int(abs(cos(s*2*pi.n()/36))*(s/77)**(1/2.0)*16)+2#int(sum(Integer(k+d+1).digits(5)))+1
        #vVolume =  int(abs(sin(s*2*pi.n()/36))*(s/77)**(1/2.0)*8)+2#int(sum(Integer(k+d+1).digits(2)))+1        
        t = int(s*D1)
        a,b = -1,1
        x,y = k,t
        p = nth_prime(d+3)
        dig = Integer(4**t).digits(11,padto=len(durs))#( -y**2+x**3+a*x+b)%p+1 #int(abs(cos((s*k)*2*pi.n()/36)*8))+2
        dig.reverse()
        print("dig",dig)
        #vPitch = dig[d]+1
        vpitches[-1].append(vPitch)
        #vVolume = int(sin((s+k)**(3.0/2.0)*2*pi.n()/36)*4)+4+2
        print("vPitch,vVolume",vPitch,vVolume)
        #v = int(duration1*by)
        #v = sigma(euler_phi(k+d+1))
        print(v)
        nextPitch = findBestMatches(nbrsPitch,startPitch,n_neighbors=min(len(pitches),vPitch))
        nextVolume = findBestMatches(nbrsVolume,startVolume,n_neighbors=min(len(volumes),vVolume))
        nextPause = findBestMatches(nbrsPause,startPause,n_neighbors=min(len(isPauses),2))
        pitches1.extend([nextPitch[-1]])
        volumes1.extend([nextVolume[-1]])
        pa0 = nextPause[0]
        #isPause = isPauses[pa0] or ( d > s**2*16.0/35**2  and s <= 35 ) or (s>35 and d>-8/35*s+24)
        #isPause = isPauses[pa0] or (d>-2.889908+4.251318*s-0.3608946*s**2+0.01179963*s**3-0.0001617849*s**4+7.823284*10**-7*s**5)
        #t = abs(s-77)
        t = s
        isPause = d>8-(not (s>=30 and s<=50))*abs(s-30)*2/60*8 #( d >= func(s)) #float(s**2*12.0/35**2)  and s <= 35 ) or (s>35 and d>float(-8.0/35*s+24))
        pauses1.append(isPause)
        pauses1.append(nextPause[1])
        p0 = pitches1.pop(0)
        v0 = volumes1.pop(0)
        #pa0 = pauses1.pop(0)
        pitch = pitches[p0]
        volume = volumes[v0]
        
        withBuffer = False
        if len(pitchBuffer)<=lenBuffers[0] and withBuffer:
            pitchBuffer.append(pitch)
        if len(pitchBuffer)>lenBuffers[0] and withBuffer:
            #lenBuffers.append(lenBuffers.pop(0))
            p = pitchBuffer.pop(0)
            m = int(floor(median(pitchBuffer)))
            octave = p//12
            pitchMod12 = p%12
            print(pitchMod12,m)
            pit = 12*octave + (7*pitchMod12+m)%12 #m-abs(pitch-m) if m > pitch else m+abs(pitch-m)
            pitchBuffer.append(pitch)
            pitch = pit
            
        #startPitch = chPitch[p0]
        #startVolume = chVolume[v0]
        #startPause = chPause[pa0]
        note1 = pitch,float(duration1*1.0),volume,isPause
        print(note1)
        pitchSet.add(pitch)
        iinds[d].append(note1)

    
print(iinds)   

midiInstrumentNumber = 51 # synth strings
instr = len(durs)*[0] #range(len(durs)) #(len(durs))*[0] # 51 synth strings
#instr.extend((nVoices//2)*[0]) # piano
#instr.extend((nVoices//4)*[0]) # sine wave (81)


fn1 = "./midi/experiment-"+str(gr).replace(" ","-")+"graph_"+str(N)+"-"+str(by)+"_"+str(nVoices)+".mid"
#fn = "../midi/experiment_in_intensity_for_piano.mid"
fn = fn1

writePitches(fn=fn,inds=iinds,tempo=80,instrument=instr,add21=True,start_at= len(durs)*[0],durationsInQuarterNotes=False)

print(vpitches)
print(pitchSet)

[<music21.pitch.Pitch E-0>, <music21.pitch.Pitch E0>, <music21.pitch.Pitch F0>, <music21.pitch.Pitch F#0>, <music21.pitch.Pitch G0>, <music21.pitch.Pitch G#0>, <music21.pitch.Pitch A0>, <music21.pitch.Pitch B-0>, <music21.pitch.Pitch B0>, <music21.pitch.Pitch C1>, <music21.pitch.Pitch C#1>, <music21.pitch.Pitch D1>, <music21.pitch.Pitch E-1>, <music21.pitch.Pitch E1>, <music21.pitch.Pitch F1>, <music21.pitch.Pitch F#1>, <music21.pitch.Pitch G1>, <music21.pitch.Pitch G#1>, <music21.pitch.Pitch A1>, <music21.pitch.Pitch B-1>, <music21.pitch.Pitch B1>, <music21.pitch.Pitch C2>, <music21.pitch.Pitch C#2>, <music21.pitch.Pitch D2>, <music21.pitch.Pitch E-2>, <music21.pitch.Pitch E2>, <music21.pitch.Pitch F2>, <music21.pitch.Pitch F#2>, <music21.pitch.Pitch G2>, <music21.pitch.Pitch G#2>, <music21.pitch.Pitch A2>, <music21.pitch.Pitch B-2>, <music21.pitch.Pitch B2>, <music21.pitch.Pitch C3>, <music21.pitch.Pitch C#3>, <music21.pitch.Pitch D3>, <music21.pitch.Pitch E-3>, <music21.pitch.Pitch 

dig [1, 1, 8, 8, 2, 2, 2, 6, 4, 6, 5, 0, 7, 8, 3, 5, 9, 5, 1, 3, 9, 3, 4, 7, 4, 6, 10, 4, 0, 5, 2, 2, 1, 2, 3, 4, 1, 5, 1, 5, 7, 2, 9, 7, 2, 4, 8, 8, 3, 10, 0, 5, 7, 8, 3, 2, 2, 3, 5, 5, 3, 5, 10, 9, 5, 7, 9, 4]
vPitch,vVolume 2 5
(0, 0, 1, 1)
(27, 0.25, 80, False)
dig [1, 7, 6, 7, 10, 2, 2, 8, 3, 7, 4, 4, 0, 3, 1, 1, 5, 8, 4, 10, 6, 5, 10, 8, 8, 8, 1, 0, 9, 7, 6, 2, 0, 8, 3, 10, 0, 1, 5, 2, 2, 6, 2, 0, 5, 5, 10, 8, 1, 7, 6, 8, 3, 3, 0, 7, 2, 4, 0, 10, 8, 1, 7, 8, 9, 3, 4, 6, 9]
vPitch,vVolume 3 6
(0, 0, 1, 1)
(34, 0.25, 96, False)
dig [2, 5, 0, 8, 5, 9, 3, 0, 1, 3, 8, 3, 9, 4, 5, 7, 2, 4, 3, 2, 4, 5, 7, 7, 8, 7, 8, 6, 3, 1, 0, 0, 0, 1, 1, 7, 6, 2, 1, 6, 2, 7, 10, 10, 8, 0, 7, 6, 9, 5, 0, 9, 0, 8, 4, 10, 5, 4, 10, 4, 6, 9, 5, 3, 9, 5, 10, 7, 10, 1]
vPitch,vVolume 4 4
(0, 0, 1, 1)
(39, 0.25, 64, False)
dig [9, 9, 3, 1, 1, 4, 1, 0, 5, 4, 0, 4, 4, 7, 0, 6, 9, 6, 1, 9, 7, 0, 8, 9, 1, 9, 1, 3, 1, 4, 0, 0, 0, 4, 6, 8, 2, 8, 6, 2, 10, 9, 10, 9, 10, 2, 8, 5, 4, 9, 3, 3, 3, 0, 8, 8, 10, 8, 8, 7

(0, 0, 1, 1)
(15, 0.125, 96, False)
dig [3, 6, 4, 8, 2, 4, 3, 7, 0, 2, 10, 0, 2, 6, 4, 9, 4, 0, 0, 5, 5, 4, 1, 8, 8, 4, 5, 1, 5, 7, 8, 8, 8, 4, 1, 9, 8, 8, 9, 5, 4, 0, 4, 10, 1, 7, 10, 7, 3, 3, 8, 3, 2, 5, 4, 4, 1, 2, 8, 10, 5, 3, 9, 2, 4, 8, 3, 1, 3, 2, 7, 2, 3, 1, 5, 3, 6, 3, 10, 8, 2, 4, 5, 3, 1, 3, 1, 0, 4, 7, 1, 8, 8, 4, 9, 6, 9, 10, 1, 8, 1, 7, 6, 5, 1, 9, 10, 4, 5, 8, 8, 8, 10, 2, 6, 0, 4, 7, 5, 8, 0, 1, 2, 4, 5, 2, 5, 3, 0, 6, 3, 3, 7, 2, 7, 10, 3, 3, 2, 1, 6, 10, 5, 8, 1]
vPitch,vVolume 2 6
(0, 0, 1, 1)
(27, 0.125, 96, False)
dig [1, 3, 3, 7, 10, 9, 6, 3, 6, 1, 0, 7, 0, 10, 3, 8, 4, 5, 0, 1, 10, 10, 5, 7, 2, 0, 6, 9, 6, 0, 9, 2, 2, 0, 5, 7, 6, 2, 2, 4, 10, 5, 1, 8, 7, 6, 9, 9, 7, 2, 4, 0, 1, 9, 10, 6, 5, 5, 0, 2, 8, 10, 4, 3, 9, 8, 0, 1, 5, 1, 10, 6, 9, 1, 5, 10, 3, 3, 4, 9, 10, 9, 6, 10, 1, 5, 1, 4, 1, 7, 6, 7, 2, 0, 8, 5, 5, 6, 7, 6, 10, 6, 8, 3, 9, 7, 6, 8, 7, 1, 2, 2, 2, 7, 10, 2, 1, 7, 8, 0, 10, 0, 4, 9, 6, 9, 9, 10, 1, 2, 3, 2, 3, 6, 10, 9, 8, 2, 1, 8, 6, 5, 9, 0, 10, 4]

(27, 0.125, 80, False)
dig [1, 3, 3, 10, 5, 1, 6, 4, 4, 3, 3, 8, 4, 8, 2, 0, 0, 10, 2, 1, 3, 10, 5, 5, 0, 9, 8, 3, 10, 10, 10, 7, 9, 6, 3, 3, 6, 9, 10, 3, 4, 0, 7, 5, 6, 4, 2, 8, 4, 0, 6, 8, 0, 1, 9, 2, 7, 6, 5, 6, 6, 3, 9, 4, 9, 3, 10, 3, 0, 1, 7, 2, 9, 6, 5, 5, 0, 3, 9, 1, 6, 6, 5, 6, 2, 10, 3, 3, 2, 2, 6, 8, 1, 2, 9, 6, 7, 1, 8, 2, 9, 3, 7, 7, 0, 0, 10, 10, 4, 9, 1, 3, 9, 2, 10, 5, 5, 4, 2, 2, 7, 0, 5, 0, 1, 5, 5, 6, 1, 5, 4, 10, 1, 4, 9, 5, 8, 9, 2, 7, 1, 4, 6, 8, 8, 5, 10, 10, 9, 1, 6, 7, 5, 10, 4, 2, 2, 1, 6, 10, 0, 8, 5, 0, 8, 4, 6, 5, 3, 6, 1, 4, 8, 6, 2, 8, 1, 6, 2, 2, 3, 7, 10, 6, 4, 4, 3, 5, 0, 5, 1, 2, 10, 3, 8, 8, 5, 0, 9, 7, 5, 2, 6, 3, 3, 10, 10, 1, 5, 0, 3, 1, 7, 2, 8, 6, 5, 7, 4, 3]
vPitch,vVolume 3 6
(0, 0, 1, 1)
(34, 0.125, 96, False)
dig [5, 2, 4, 8, 9, 6, 3, 6, 6, 2, 4, 0, 7, 10, 8, 0, 3, 7, 8, 5, 4, 8, 10, 9, 3, 6, 0, 4, 10, 10, 9, 9, 5, 3, 2, 3, 5, 6, 8, 2, 5, 2, 8, 0, 3, 6, 0, 0, 5, 2, 4, 10, 0, 7, 3, 10, 8, 4, 0, 4, 3, 4, 4, 8, 4, 4, 8, 1, 0, 6, 7, 0, 5, 3, 10,

(27, 0.25, 80, False)
dig [7, 6, 2, 7, 9, 7, 1, 5, 3, 9, 5, 4, 2, 10, 0, 3, 7, 8, 4, 2, 4, 3, 4, 8, 4, 6, 9, 4, 7, 8, 1, 6, 6, 6, 4, 5]
vPitch,vVolume 3 3
(0, 0, 1, 1)
(34, 0.25, 48, False)
dig [1, 0, 0, 0, 10, 5, 0, 4, 1, 8, 6, 8, 9, 2, 2, 6, 5, 4, 3, 2, 1, 5, 2, 10, 10, 2, 7, 10, 7, 9, 2, 9, 3, 6, 6, 3, 5, 3]
vPitch,vVolume 1 4
(0, 0, 1, 1)
(15, 0.25, 64, False)
dig [2, 1, 3, 2, 0, 1, 4, 1, 8, 9, 5, 6, 3, 7, 2, 0, 3, 0, 4, 9, 3, 7, 4, 7, 7, 4, 5, 4, 6, 3, 5, 5, 3, 0, 7, 10, 10, 10, 7, 9]
vPitch,vVolume 2 4
(0, 0, 1, 1)
(27, 0.5, 64, False)
dig [4, 5, 3, 0, 0, 8, 10, 0, 10, 0, 3, 2, 8, 4, 0, 2, 1, 4, 8, 2, 9, 2, 5, 9, 3, 6, 4, 6, 0, 5, 0, 4, 5, 9, 3, 10, 1, 10, 4, 2, 10, 5]
vPitch,vVolume 3 5
(0, 0, 1, 1)
(34, 0.5, 80, False)
dig [6, 5, 7, 4, 1, 1, 10, 7, 3, 6, 4, 8, 0, 1, 9, 3, 1, 0, 10, 0, 1, 4, 7, 5, 6, 2, 3, 6, 8, 7, 3, 6, 6, 5, 6, 7, 8, 9, 1, 2, 3, 2, 3]
vPitch,vVolume 1 4
(0, 0, 1, 1)
(15, 0.25, 64, False)
dig [9, 5, 2, 7, 10, 7, 9, 5, 7, 2, 3, 9, 7, 2, 7, 5, 5, 6, 3, 6, 2, 0, 8

...

times >  48 154.000000000000 1.0 96 False
writePitches 34 0.25 112 False
times >  55 155.000000000000 1.0 112 False
writePitches 39 0.25 80 False
times >  60 156.000000000000 1.0 80 False
writePitches 15 0.125 96 False
times >  36 156.500000000000 0.5 96 False
writePitches 27 0.125 96 False
times >  48 157.000000000000 0.5 96 False
writePitches 34 0.125 112 False
times >  55 157.500000000000 0.5 112 False
writePitches 39 0.125 96 False
times >  60 158.000000000000 0.5 96 False
writePitches 15 0.125 112 False
times >  36 158.500000000000 0.5 112 False
writePitches 27 0.125 112 False
times >  48 159.000000000000 0.5 112 False
writePitches 34 0.125 128 False
times >  55 159.500000000000 0.5 128 False
writePitches 39 0.125 80 False
times >  60 160.000000000000 0.5 80 False
writePitches 15 0.25 96 False
times >  36 161.000000000000 1.0 96 False
writePitches 27 0.25 96 False
times >  48 162.000000000000 1.0 96 False
writePitches 34 0.25 112 False
times >  55 163.000000000000 1.0 112 False
wr

pause - times >  15 29.0000000000000 0.5 80 True
writePitches 27 0.125 80 True
pause - times >  27 29.5000000000000 0.5 80 True
writePitches 34 0.125 96 False
times >  55 30.0000000000000 0.5 96 False
writePitches 39 0.125 48 False
times >  60 30.5000000000000 0.5 48 False
writePitches 15 0.125 64 False
times >  36 31.0000000000000 0.5 64 False
writePitches 27 0.125 64 False
times >  48 31.5000000000000 0.5 64 False
writePitches 34 0.25 80 False
times >  55 32.5000000000000 1.0 80 False
writePitches 39 0.25 64 False
times >  60 33.5000000000000 1.0 64 False
writePitches 15 0.25 80 False
times >  36 34.5000000000000 1.0 80 False
writePitches 27 0.25 80 False
times >  48 35.5000000000000 1.0 80 False
writePitches 34 0.125 96 False
times >  55 36.0000000000000 0.5 96 False
writePitches 39 0.125 64 False
times >  60 36.5000000000000 0.5 64 False
writePitches 15 0.125 80 False
times >  36 37.0000000000000 0.5 80 False
writePitches 27 0.125 80 False
times >  48 37.5000000000000 0.5 80 False


times >  48 76.0000000000000 2.0 96 False
writePitches 34 0.125 80 False
times >  55 76.5000000000000 0.5 80 False
writePitches 15 0.25 96 False
times >  36 77.5000000000000 1.0 96 False
writePitches 27 0.25 96 False
times >  48 78.5000000000000 1.0 96 False
writePitches 34 0.25 112 False
times >  55 79.5000000000000 1.0 112 False
writePitches 15 0.625 32 False
times >  36 82.0000000000000 2.5 32 False
writePitches 27 0.5 48 False
times >  48 84.0000000000000 2.0 48 False
writePitches 34 0.125 48 False
times >  55 84.5000000000000 0.5 48 False
writePitches 15 0.25 64 False
times >  36 85.5000000000000 1.0 64 False
writePitches 27 0.25 48 False
times >  48 86.5000000000000 1.0 48 False
writePitches 34 0.25 64 False
times >  55 87.5000000000000 1.0 64 False
writePitches 15 0.625 64 False
times >  36 90.0000000000000 2.5 64 False
writePitches 27 0.5 80 False
times >  48 92.0000000000000 2.0 80 False
writePitches 34 0.125 48 False
times >  55 92.5000000000000 0.5 48 False
writePitches 15 0

writePitches 27 0.125 48 True
pause - times >  27 214.500000000000 0.5 48 True
writePitches 15 0.125 64 True
pause - times >  15 215.000000000000 0.5 64 True
writePitches 27 0.125 64 True
pause - times >  27 215.500000000000 0.5 64 True
writePitches 15 0.375 80 True
pause - times >  15 217.000000000000 1.5 80 True
writePitches 27 0.25 64 True
pause - times >  27 218.000000000000 1.0 64 True
writePitches 15 0.25 80 True
pause - times >  15 219.000000000000 1.0 80 True
writePitches 27 0.375 80 True
pause - times >  27 220.500000000000 1.5 80 True
writePitches 15 0.125 96 True
pause - times >  15 221.000000000000 0.5 96 True
writePitches 27 0.125 64 True
pause - times >  27 221.500000000000 0.5 64 True
writePitches 15 0.125 80 True
pause - times >  15 222.000000000000 0.5 80 True
writePitches 27 0.125 80 True
pause - times >  27 222.500000000000 0.5 80 True
writePitches 15 0.125 96 True
pause - times >  15 223.000000000000 0.5 96 True
writePitches 27 0.125 80 True
pause - times >  27 223.

times >  79 145.500000000000 1.0 64 False
writePitches 46 0.25 80 False
times >  67 146.500000000000 1.0 80 False
writePitches 46 0.25 80 False
times >  67 147.500000000000 1.0 80 False
writePitches 51 0.25 96 False
times >  72 148.500000000000 1.0 96 False
writePitches 53 0.125 80 False
times >  74 149.000000000000 0.5 80 False
writePitches 55 0.125 96 False
times >  76 149.500000000000 0.5 96 False
writePitches 46 0.125 96 False
times >  67 150.000000000000 0.5 96 False
writePitches 51 0.125 112 False
times >  72 150.500000000000 0.5 112 False
writePitches 53 0.125 80 False
times >  74 151.000000000000 0.5 80 False
writePitches 55 0.125 96 False
times >  76 151.500000000000 0.5 96 False
writePitches 55 0.125 96 False
times >  76 152.000000000000 0.5 96 False
writePitches 51 0.125 112 False
times >  72 152.500000000000 0.5 112 False
writePitches 53 0.25 96 False
times >  74 153.500000000000 1.0 96 False
writePitches 55 0.25 112 False
times >  76 154.500000000000 1.0 112 False
writePit